$$Exercise:\;1$$

In [479]:
!pip install -q pyomo

In [480]:
from pyomo.environ import *

In [481]:
import numpy as np

In [482]:
model_ex1 = ConcreteModel()

In [483]:
N = 3 #number of decision variables 
M = 3 #number of constraints without considering the bound constraints  
lb=np.array([1,2,1])
ub=np.array([np.inf,np.inf,np.inf]) 
obj_coef_c = np.array([1,-2,3]) 
constr_coef_A=np.array([[2,1,2],[-1,1,2],[2,-3,1]])
constr_rhs_b=np.array([10,-3,-6])
row_indices=np.arange(M)
print(row_indices)
col_indices = np.arange(N)
print(col_indices)

[0 1 2]
[0 1 2]


In [484]:
model_ex1.x=Var(col_indices)
model_ex1.constraints = ConstraintList()

In [485]:
for i in col_indices:
  model_ex1.x[i].setlb(lb[i])
  model_ex1.x[i].setub(ub[i])

In [486]:
for i in row_indices:
  model_ex1.constraints.add(sum(constr_coef_A[i][j]*model_ex1.x[j] for j in col_indices) <= constr_rhs_b[i])

In [487]:
model_ex1.objective = Objective(expr =summation(obj_coef_c,model_ex1.x), sense=minimize)
model_ex1.pprint() 

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] + x[1] + 2*x[2] :  10.0 

$$1.\;Solving\;using\;glpk\;solver:$$

In [488]:
!apt-get install -y -qq glpk-utils

In [489]:
opt = SolverFactory('glpk', executable='/usr/bin/glpsol')
#.solve(model1).write()
result = opt.solve(model_ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 10
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: other
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.016651153564453125

Solver status: ok
Solver termination condition: other


$For\; the\;  'glpk\;  solver',\;  the\;  'Status'\;  is \; 'ok'\;  and\;  the\;  'Termination \; Condition' \; is \; 'other'. \; This\;  does\;  not \; provide\;  us\;  any \; meaningful\;  message \; like, \; is\;  the \; objective\;  function \; tending \; to \; +\infty \; or -\infty?\;  or \;is \;  the solution\; feasible\;  or \; not?$

$$1.\;Solving\;using\;cbc\;solver:$$

In [490]:
!apt-get install -y -qq coinor-cbc

In [491]:
opt_cbc = SolverFactory('cbc')

In [492]:
result = opt_cbc.solve(model_ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.022086620330810547

Solver status: warning
Solver termination condition: infeasible
Solver time: 0.022086620330810547


$For\; the\;  'cbc\;  solver',\;  the\;  'Status'\;  is \; 'warning'\;  and\;  the\;  'Termination \; Condition' \; is \; 'infeasible'. \;Here,\; the\;status\;and\;Termination\;Condition\;implies\;that,\;some\;constraints\;are\;contradictory\;to\;each\;other\;and \;hence, \;there \;is\; no\; feasible \;solution\; which\;directly\;  implies\; no \;optimal\; solution.$


$2.\;(a):\;Changing\; the\; optimization\; problem \;to\; a \;maximization \;problem\; and \;re-solving.$

In [493]:
model_ex1.objective.set_sense(maximize)

In [494]:
model_ex1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] + x[1] + 2*x[2] :  10.0 

In [495]:
result = opt_cbc.solve(model_ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: None
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: maximize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.01789689064025879

Solver status: warning
Solver termination condition: infeasible
Solver time: 0.01789689064025879


$Here,\;  the\;  'Status'\;  is \; 'warning'\;  and\;  the\;  'Termination \; Condition' \; is \; 'infeasible'.\;The \;solution\; is \;infeasible \;because,\; if \;we \;multiply\; 2\; in\; both \;sides\; of \;the\; 2nd \;constraint\; and \;then \;add \;that \;to\; the \;1st \;constraint,\;then \;we \;will\; obtain \;the \;following\;inequality: \; 3x_2+6x_3\leq4 \;which \;is\; not\; possible \;since\; the\; lower\; bounds\; of\; x_2\;and \;x_3\; are\; 2\; and\; 1\;respectively\; and\; due \;to \;that, \;the \;lowest\; value\; of\; 3x_2+6x_3 \;can\; be\; 12\geq4.\; That's \;why,\; the \;model\; do \;not\; posses\; any \;feasible\; solution.$

$2.\;(b):\;Adding\;an\;upper\;bound\;on\;x_2\;as\;x_2\leq8\;and\;re-solving.$

In [496]:
model_ex1.objective.set_sense(minimize)
model_ex1.x[1].setub(8)

In [497]:
model_ex1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :     8 : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] + x[1] + 2*x[2] :  10.0 

In [498]:
result = opt_cbc.solve(model_temp1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: None
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 4
  Number of nonzeros: 3
  Sense: maximize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.020777225494384766

Solver status: warning
Solver termination condition: infeasible
Solver time: 0.020777225494384766


$Here\;also,\;  the\;  'Status'\;  is \; 'warning'\;  and\;  the\;  'Termination \; Condition' \; is \; 'infeasible'.\;The \;solution\; is \;infeasible \;because,\; if \;we \;multiply\; 2\; in\; both \;sides\; of \;the\; 2nd \;constraint\; and \;then \;add \;that \;to\; the \;3rd \;constraint,\;then \;we \;will\; obtain \;the \;following\;inequality: \; -x_2+5x_3\leq-12 \;which \;is\; not\; possible \;since\; the\; lower\; bound\; of\; x_3\; is\; 1\; and\;upper\; bound\; of\; x_2\; is\; 8\; and\; due \;to \;that, \;the \;lowest\; value\; of\;-x_2+5x_3 \;can\; be\; -3\geq-12.\; That's \;why,\; the \;model\; do \;not\; posses\; any \;feasible\; solution.$

$2.\;(c):\;Adding\;a\;new\;constraint\;-x_2+x_3\leq9\;and\;re-solving.$

In [499]:
model_ex1.x[1].setub(np.inf)
new_constr_coef = [0,-1,1]
new_constr_rhs = 9
model_ex1.constraints.add(sum(new_constr_coef[j]*model_ex1.x[j] for j in col_indices) <= new_constr_rhs)
model_ex1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraints : Size=4, Index=constraints_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] + x[1] + 2*x[2] :  10

In [500]:
result = opt_cbc.solve(model_ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.01986098289489746

Solver status: warning
Solver termination condition: infeasible
Solver time: 0.01986098289489746


$Here,\;  the\;  'Status'\;  is \; 'warning'\;  and\;  the\;  'Termination \; Condition' \; is \; 'infeasible'.\;The \;solution\; is \;infeasible \;because,\; if \;we \;multiply\; 2\; in\; both \;sides\; of \;the\; 2nd \;constraint\; and \;then \;add \;that \;to\; the \;1st \;constraint,\;then \;we \;will\; obtain \;the \;following\;inequality: \; 3x_2+6x_3\leq4 \;which \;is\; not\; possible \;since\; the\; lower\; bounds\; of\; x_2\;and \;x_3\; are\; 2\; and\; 1\;respectively\; and\; due \;to \;that, \;the \;lowest\; value\; of\; 3x_2+6x_3 \;can\; be\; 12\geq4.\; That's \;why,\; the \;model\; do \;not\; posses\; any \;feasible\; solution.$

$2.\;(d):\;Deactivating\; the \;objective\; of \;problem (OP) \;and\; constructing\; a\; new \;objective \;of\; the \;form\;
min\; x_1 + x_2 + x_3 and deactivating\; the \;constraint\; 2x_1 + x_2 + 2x_3 ≤ 10 \;and\; adding\; a \;different\; constraint\; x1 + x2 ≥ 45\;and\;re-solving.$

In [501]:
model_ex1.constraints[4].deactivate()
model_ex1.objective.deactivate()
obj_coef_c_new = np.array([1,1,1]) 
model_ex1.new_objective = Objective(expr = summation(obj_coef_c_new,model_ex1.x) )
model_ex1.constraints[1].deactivate()
model_ex1.constraints.add(expr = model_ex1.x[0]+model_ex1.x[1] >= 45)

In [502]:
model_ex1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

2 Objective Declarations
    new_objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] + x[1] + x[2]
    objective : Size=1, Index=None, Active=False
        Key  : Active : Sense    : Expression
        None :  False : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraints : S

In [503]:
result = opt_cbc.solve(model_ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)


Problem: 
- Name: unknown
  Lower bound: 46.0
  Upper bound: 46.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 2
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 2
  Error rc: 0
  Time: 0.02979564666748047
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal
Solver time: 0.02979564666748047


$Here,\;  the\;  'Status'\;  is \; 'ok'\;  and\;  the\;  'Termination \; Condition' \; is \; 'optimal'.\;Hence,\;all\;constraints\;are \;simultaneously\;satisfied.$

$The \;values\; of\; the\; Objective\; function, \;variables\; and \;the \;active\; status\; of \;constraints\; are \;as \;follows:$

In [504]:
# display solution
print('\nObjective = ', model_ex1.new_objective())

print('\nDecision Variables')
for i in col_indices:
  print('x'+str(i)+'= ', model_ex1.x[i].value) 
print('\nConstraints')
model_ex1.constraints.display()


Objective =  46.0

Decision Variables
x0=  25.6
x1=  19.4
x2=  1.0

Constraints
constraints : Size=5
    Key : Lower : Body               : Upper
      2 :  None : -4.200000000000003 :  -3.0
      3 :  None : -5.999999999999993 :  -6.0
      5 :  45.0 :               45.0 :  None


$The\;third\;and\;fifth\;constraints\;i.e\; 2x_1 - 3x_2+ x_3\leq -6.0\;and\;45.0 \leq x_1 + x_2\;are\;active.$